How to use gulp?

In [ ]:
var execSync = require('child_process').execSync;
try {
    require.resolve('gulp');
    require.resolve('gulp-runtime');
    require.resolve('gulp-replace');
    require.resolve('gulp-tap');
} catch (e) {
    execSync('npm install "github:gulpjs/gulp#4.0" gulp-runtime gulp-replace gulp-tap');
}
var gulp = require('gulp');
var tap = require('gulp-tap');
var replace = require('gulp-replace');

// rename the project
gulp.task('build :src :dest', function (files, project) {
    return gulp
        .src(files, {cwd: project})
        .pipe(replace('appId: \'my-app-id\'', 'appId: \'jupytangular-module-server\''))
        .pipe(tap(function(file) {
            console.log(file.contents.toString());
        }))
});

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Documents/universal';

gulp.task('default', () => gulp.task(['build :src :dest'])(['**/app.browser.module.ts'], project));

$$.async()
try {
gulp.series(() => gulp.task(['default'])(), function (done) {
    done();
    $$.sendResult();
})();
} catch (e) {
    $$.sendError(e);
}

How to convert a gulp task to a Promise?

How to use a Promise as a gulp task?



In [ ]:
var gulp = require('gulp');

// I wish Linus Torvolds had absolute power over everything javascript(+derivatives) and he could just do a massive purge of all the bullshit.
var tasksToPromise = (tasks) => {
    try {
        return new Promise((resolve, reject) => {
            gulp.series.apply(gulp, tasks.concat([function (done) {
                resolve(done());
            }]))();
        });
    } catch(e) {
        reject(e);
    }
};
module.exports = tasksToPromise;
tasksToPromise;



In [ ]:
var fs = require('fs');
var path = require('path');
var replace = require('gulp-replace');
var gulp = require('gulp');

// add automock to project for callback services
var client, renderer, server, core;

var applyUniversal = (project) => {
    // gulp.task('theme',
    //    gulp.src('./some/dir/**/*.js', {cwd: 'path/to/project'})
    //        .src('./specific-file.ts')) // already within cwd in chain?

    // TODO: copy auth, resolve dependencies using file system script, generate code below
    
    // replace package.json
    gulp.task('project relatives',
        gulp.series(projectRelatives(project)))
    // in *.js Can't resolve 'socket.io-client/package'
    // in *.ts Cannot find name 'JwtHelper' -> trace back to import statements?
    
    
    gulp.task('server module', function() {
        return gulp.src(['**/server/app/app.server.module.js'], {cwd: project})
            // add mock socket server to server/render-server or app.server.module
        .pipe(replace('import { AppComponent } from \'./app.component\'', `
import { AppComponent } from './app.component';
import { AuthService } from '../../client/app/auth/auth.service';
import { LogService } from '../../imports/log.service';
import { AuthManager } from '../../client/app/auth/auth.manager';
import { JwtHelper } from 'angular2-jwt/angular2-jwt';
import { MailgunValidatorService } from '../../client/app/auth/mailgun-validate.service';
`))
   // TODO: pipe server.ts to add log provider for log service    
            .pipe(replace(
             /\)\],\s*AppModule\s*,providers\s*:\s*\[/ig,
              `)],
        AppModule
    ],
    providers: [
        {
            provide: AuthService,
            useValue: require('./sockify-server').sockifyRequire(AuthService, 'AuthService')
        },
        {
            provide: PlatformLocation,
            useValue: require('./sockify-server').sockifyRequire(PlatformLocation, 'PlatformLocation')
        },
        {
            provide: LogService,
            useValue: require('./sockify-server').sockifyRequire(LogService, 'LogService')
        },
        {
            provide: AuthGuard,
            useValue: require('./sockify-server').sockifyRequire(AuthGuard, 'AuthGuard')
        },
        AuthManager, JwtHelper, MailgunValidatorService,

    `))
            .pipe(gulp.dest(project))
    });
    
    // add socket-client provider for auth module to app.browser.module
    gulp.task('browser module', function() {
      return gulp.src(['**/client/app/app.browser.module.ts'], {cwd: project})
        .pipe(replace('import { AppComponent } from \'./app.component\'', `
import { AppComponent } from './app.component';
import { AuthService } from './auth/auth.service';
import { PlatformLocation } from '@angular/common';
import { LogService } from '../../imports/log.service';
import { AuthManager } from './auth/auth.manager';
import { JwtHelper } from 'angular2-jwt/angular2-jwt';
import { MailgunValidatorService } from './auth/mailgun-validate.service';
`))
        .pipe(replace(
          /\)\],\s*AppModule/ig,
          `)],
        AppModule,
    ],
    providers: [ AuthManager, JwtHelper, MailgunValidatorService,
        {
            provide: AuthService,
            useValue: require('./sockify-client').sockifyClient(AuthService, 'AuthService'),
        },
        {
            provide: PlatformLocation,
            useValue: require('./sockify-client').sockifyClient(PlatformLocation, 'PlatformLocation'),
        },
`))
        .pipe(gulp.dest(project))
    });
    
    // replace routes with lazy modules from other projects
    gulp.task('app router', function() {
      return gulp.src(['**/client/app/app.routes.ts'], {cwd: project})
        .pipe(replace(
         /children: \[/ig, 
          `children: [
            {
                path: 'auth',
                loadChildren: './auth/auth.module#AuthModule'
            },
`))
        .pipe(gulp.dest(project))
    });
    
    // TODO: try new versions of packages to act like greenkeeper.io
    
    // gulp apply everything
    gulp.task('apply universal',
        gulp.series('server module',
                    'browser module'
                    'app router'));
    
    return tasksToPromise(['apply universal'], []);
};
applyUniversal;



Apply universal to angular project?

In [ ]:

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Documents/universal';
if(!fs.existsSync(project)) {
    fs.mkdirSync(project);
}

$$.async()
applyUniversal(project)
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));



Fix project relatives as a service for updating files when they change?



In [ ]:
var execSync = require('child_process').execSync;
try {
    require.resolve('gulp');
    require.resolve('gulp-watch');
} catch (e) {
    execSync('npm install "github:gulpjs/gulp.git#4.0" gulp-watch');
}
var gulp = require('gulp'),
    watch = require('gulp-watch');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Documents/universal';

var searching = false;;
gulp.task('watch for changes', function () {
    // Callback mode, useful if any plugin in the pipeline depends on the `end`/`flush` event 
    return watch('src/**/*.ts', {
        cwd: project,
        ignoreInitial: true,
        read: false,
        readDelay: 100
    }, function () {
        if(!searching) {
            searching = true;
            projectRelatives(project).then(() => { searching = false });
        }
    });
});

$$.async();
gulp.task(['watch for changes'])();



Search notebooks for a string using gulp?



In [3]:
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/jupytangular2';

var gulp = require('gulp');
var tap = require('gulp-tap');
var JSONStream = require('JSONStream');
var Duplex = require('stream').Duplex;

function bufferToStream(buffer) {  
    let stream = new Duplex();
    stream.push(buffer);
    stream.push(null);
    return stream;  
}

var cells;
gulp.task('search notebooks', function () {
    cells = [];
    return gulp.src(['**/*.ipynb', '!**/node_modules/**', '!**/\\.*'], {cwd: project})
        .pipe(tap(file => {
            var matches = bufferToStream(file.contents)
                .pipe(JSONStream.parse([true, {emitPath: true}]));
            matches.on('data', match => {
                if(match.path[0] === 'cells') {
                    if(match.value.source.join('').match(new RegExp('regex', 'ig'))) {
                        cells[cells.length] = file.path 
                            + ':\n\n```\n' + match.value.source.join('') 
                            + '\n```\n\n';
                    }
                }
            });
        }));
});

$$.async();
gulp.series('search notebooks', function (done) {
    done();
    $$.mime({'text/markdown': cells.join('\n')});
})();


/Users/briancullinan/jupytangular2/Core/interpret all notebooks.ipynb:

```
// How to flatten and array in javascript?
var flatten = arr => arr.reduce((acc, val) => 
      acc.concat(Array.isArray(val) ? flatten(val) : val), []);

Array.prototype.flatten = function() {return flatten(this)};

// How to escape a string for regex?
function escapeRegExp(str) {
  return str.replace(/[\-\[\]\/\{\}\(\)\*\+\?\.\\\^\$\|]/g, "\\$&");
}



```


/Users/briancullinan/jupytangular2/Core/interpret questions.ipynb:

```
// TODO: use the m flag option for regexp?
var re = new RegExp('(^|\\n).*?\\?(\\s+|$)', 'ig');
var re2 = new RegExp('(^|\\n)//.*\\?(\\s+|$)', 'ig')

var accumulateMarkdown = (cells) => {
    var counter = 0, prev = [];
    return cells.reduce((md, c) => {
        counter++;
        var source = c.source.join('');
        if (c.cell_type == 'markdown') {
            prev.push(source);
            return md;
        } else if(c.cell_type !== 'code') {
            return md;
        }
        // TODO: improve the counter
        var cell = {code: source, markdown: prev, from: counter-1, to: counter, language: c.language};
        prev = [];
        md.push(cell);
        return md;
    }, []);
};

// How to convert a string to an Array of RegEx matches
var regexToArray = (ex, str, i = 0) => {
    var co = []; var m;
    while ((m = re.exec(str)) && co.push(m[i]));
    return co;
};
(regexToArray);
```


/Users/briancullinan/jupytangular2/Core/interpret questions.ipynb:

```

// TODO: interpret markdown leading up to code results and find the resulting function in the list
// for now, the boring solution is to assume all markdown output is a question?

// How to store code markdown results for later use?
if (typeof cellCache == 'undefined') { var cellCache = []; }
if (typeof cacheIds == 'undefined') { var cacheIds = {}; }
(cacheCells = ((cells, notebook) => {
    var filename = path.basename(notebook);
    var newCache = accumulateMarkdown(cells);
    newCache.forEach((c, i) => {
        if(typeof cacheIds[filename+'['+i+']'] !== 'undefined') {
            return;
        }
        var questions = regexToArray(re, c.markdown)
            .concat(regexToArray(re, c.source)
                    .map(r => r.replace(/how to|?/ig, '')));
        cacheIds[filename+'['+i+']'] = Object.assign({}, c, {
            id: filename+'['+i+']',
            filename: notebook,
            questions: questions,
            notebook: filename,
            function: global[filename+'['+i+']']
        });
        questions.forEach((q) => cellCache.push({
            id: filename+'['+i+']',
            question: q
        }));
    });
    return cellCache;
}));


```


/Users/briancullinan/jupytangular2/Frameworks/gulp.ipynb:

```
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/jupytangular2';

var gulp = require('gulp');
var tap = require('gulp-tap');
var JSONStream = require('JSONStream');
var Duplex = require('stream').Duplex;

function bufferToStream(buffer) {  
    let stream = new Duplex();
    stream.push(buffer);
    stream.push(null);
    return stream;  
}

var cells;
gulp.task('search notebooks', function () {
    cells = [];
    return gulp.src(['**/*.ipynb', '!**/node_modules/**', '!**/\\.*'], {cwd: project})
        .pipe(tap(file => {
            var matches = bufferToStream(file.contents)
                .pipe(JSONStream.parse([true, {emitPath: true}]));
            matches.on('data', match => {
                if(match.path[0] === 'cells') {
                    if(match.value.source.join('').match(new RegExp('regex', 'ig'))) {
                        cells[cells.length] = file.path 
                            + ':\n\n```\n' + match.value.source.join('') 
                            + '\n```\n\n';
                    }
                }
            });
        }));
});

$$.async();
gulp.series('search notebooks', function (done) {
    done();
    $$.mime({'text/markdown': cells.join('\n')});
})();

```


/Users/briancullinan/jupytangular2/Frameworks/ngx-translate.ipynb:

```
var execSync = require('child_process').execSync;
try {
    require.resolve('glob');
} catch (e) {
    execSync('npm install glob');
}
var glob = require('glob');
var path = require('path');
var fs = require('fs');
var translateRegex = (/[\{"]([^><"]*?)\s*\|\s*translate\s*(:\s*[^><"\}]*?)?["\}]/ig);
var cwd = '/Users/briancullinan/Documents/portal/src/';
//var cwd = 'C:\\Users\\brian.cullinan\\Documents\\portal\\src\\';
var files = glob.sync('**/*component.html', {
    ignore: '**/temp-demo-pages/**',
    cwd: cwd
});
var translations = files.map((f, i) => {
    var html = fs.readFileSync(path.join(cwd, f)).toString();
    let r;
    var translateStrings = [];
    while((r = translateRegex.exec(html)) !== null) {
        translateStrings[translateStrings.length] = r[1].trim().replace(/[\'{}\s]/ig, '');
    }
    var desiredKey = f
                .replace('app/', '')
                .replace(/\//ig, '.')
                .replace('.component.html', '')
                .replace(/-/ig, '').split('.');
    desiredKey.pop();
    desiredKey = desiredKey.join('.').toUpperCase();
    var result = {};
    result[desiredKey] = translateStrings;
    return result;
});

var translationKeys = translations.reduce((obj, tr) => {
    var key = Object.keys(tr).pop();
    if(typeof obj[key] !== 'undefined') {
        obj[key] = obj[key].concat(tr[key]);
    } else {
        obj[key] = tr[key];
    }
    return obj;
}, {});

function findMisplaced(obj, parentKey)
{
    for (var k in obj)
    {
        if (typeof obj[k] == "object" && obj[k] !== null) {
            var newParent = typeof parentKey !== 'undefined' ? (parentKey + '.' + k) : k;
            findMisplaced(obj[k], newParent);
        } else if(typeof obj[k] == "string") {
            if(obj[k].substr(0, parentKey.length) != parentKey) {
                console.log('Misplaced key: ' + obj[k] + ' in ' + parentKey);
            }
        }
    }
}

findMisplaced(translationKeys);

// get a long list of existing keys from en.js
var enJson = JSON.parse(fs.readFileSync(path.join(cwd, 'assets', 'i18n', 'en.json')).toString());


function flattenAllKeys(obj, parentKey)
{
    var result = [];
    for (var k in obj)
    {
        if (typeof obj[k] == "object" && obj[k] !== null) {
            var newParent = typeof parentKey !== 'undefined' ? (parentKey + '.' + k) : k;
            result = result.concat(flattenAllKeys(obj[k], newParent));
        } else if(typeof obj[k] == "string"
                  && parentKey.indexOf('TEMPDEMOPAGES') === -1) {
            result[result.length] = parentKey + '.' + k;
        }
    }
    return result;
}

var allENKeys = flattenAllKeys(enJson);

// find unused keys
function getUnused() {
    allENKeys.forEach(k => {
        var parentKey = k.split('.');
        parentKey.pop();
        parentKey = parentKey.join('.');
        if(typeof translationKeys[parentKey] === 'undefined' ||
            translationKeys[parentKey].indexOf(k) === -1) {
            console.log('Unused key: ' + k);
        }
    });
}

function getMissing() {
    for(var k in translationKeys) {
        if(typeof translationKeys[k] !== 'undefined') {
            translationKeys[k].forEach(i => {
                if(allENKeys.indexOf(i) === -1) {
                    console.log('Missing key: ' + i);
                }
            });
        }
    }
}

getMissing();
getUnused();


// find strings of text in html files
var htmlWordRegex = (/>[^><]*?(\b[^><]*\b)+[^><]*?</ig)
var attrWordRegex = (/(placeholder|title|alt)\s*=\s*["]([^"]*)["]/ig);
var needTranslations = files.map((f, i) => {
    var html = fs.readFileSync(path.join(cwd, f)).toString();
    let r;
    var needTranslations = [];
    while((r = htmlWordRegex.exec(html)) !== null) {
        if(r[1].trim() !== '' && r[1].match(/\|\s*translate/ig) === null) {
            needTranslations[needTranslations.length] = r[1].trim();
        }
    }
    while((r = attrWordRegex.exec(html)) !== null) {
        if(r[2].match(/\|\s*translate/ig) === null) {
            needTranslations[needTranslations.length] = r[2].trim();
        }
    }
    return {file: f, texts: needTranslations};
}).filter(t => t.texts.length > 0);

needTranslations.forEach(t => {
    console.log('Needs translating: ' + t.texts + ' in ' + t.file);
});



```


/Users/briancullinan/jupytangular2/Utilities/diff.ipynb:

```
// npm install diff --save
var importer = require('../Core')
var JsDiff = require('diff');
var prism = require('prismjs');
require('prismjs/plugins/keep-markup/prism-keep-markup');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/universal';

var htmlEntities = (str) => str.replace(/[\u00A0-\u9999<>\&]/gim, i => '&#'+i.charCodeAt(0)+';');

$$.async();
var left, right;
importer.interpretAll(['search notebook component', 'files in project'])
    .then(r => {
        left = r[0].code;
        var listInProject = r[1].runInNewContext({});
        return listInProject(project, '**/search.component.ts')
    })
    .then(r => {
        right = fs.readFileSync(r[0]).toString();
        var diff = JsDiff.diffWords(left, right);
        var code = '';
        for (var i=0; i < diff.length; i++) {
            if (diff[i].added && diff[i + 1] && diff[i + 1].removed) {
                var swap = diff[i];
                diff[i] = diff[i + 1];
                diff[i + 1] = swap;
            }

            if (diff[i].removed) {
                code += '<span class="del">' + diff[i].value + '</span>';
            } else if (diff[i].added) {
                code += '<span class="ins">' + diff[i].value + '</span>';
            } else {
                code += '<span>' + diff[i].value + '</span>';
            }
        }
        code = Prism.highlight(code, Prism.languages.javascript)
        $$.html(`
<style>

.token.comment,
.token.prolog,
.token.doctype,
.token.cdata {
    color: slategray;
}

.token.punctuation {
    color: #999;
}

.namespace {
    opacity: .7;
}

.token.property,
.token.tag,
.token.boolean,
.token.number,
.token.constant,
.token.symbol,
.token.deleted {
    color: #905;
}

.token.selector,
.token.attr-name,
.token.string,
.token.char,
.token.builtin,
.token.inserted {
    color: #690;
}

.token.operator,
.token.entity,
.token.url,
.language-css .token.string,
.style .token.string {
    color: #a67f59;
    background: hsla(0, 0%, 100%, .5);
}

.token.atrule,
.token.attr-value,
.token.keyword {
    color: #07a;
}

.token.function {
    color: #DD4A68;
}

.token.regex,
.token.important,
.token.variable {
    color: #e90;
}

.token.important,
.token.bold {
    font-weight: bold;
}

.token.italic {
    font-style: italic;
}

.token.entity {
    cursor: help;
}
.del {
    text-decoration: none;
    background: #fadad7;
}
.ins {
    background: #eaf2c2;
    text-decoration: none;
}
</style>
<pre>` + code + `</pre>`);
    })
    .catch(e => $$.sendError(e));


```


/Users/briancullinan/jupytangular2/Utilities/file system.ipynb:

```
De-lint notebook cells automatically?

Find too long of functions using regexp?
(?=\{((?:[^{}]++|\{(?1)\})++)\})

Export all notebooks to a structured folder where each cell has it's own file for linting?

Import files back in to cells? (two-way workflow)


```


/Users/briancullinan/jupytangular2/Utilities/file system.ipynb:

```
var importer = require('../Core');
var fs = require('fs');
var path = require('path');
var gulp = require('gulp');
var tap = require('gulp-tap');
var XRegExp = require('xregexp');

// How to convert a string to an Array of RegEx matches
var regexToArray = (ex, str, i = 0) => {
    var co = []; var m;
    while ((m = re.exec(str)) && co.push(m[i]));
    return co;
};

var matchCurlyBraces = (contents) => {
    var newMatches = XRegExp.matchRecursive(contents, '{', '}', 'gi', {
        valueNames: ['literal', null, 'value', null]
    }).filter(m => m.name === 'value');
    //console.log(newMatches);
    return newMatches.reduce(
        (arr, m) => arr.concat(matchCurlyBraces(m.value)),
        newMatches);
}

var tasksToPromise;
var findLongFunctions = (project) => {
    var results = [];
    gulp.task('find long functions', function () {
        return gulp.src('**/*.+(ts|js|cs)', {
            ignore: ['**/node_modules/**',
                     '**/dist/**',
                     '**/typings/**',
                    '**/packages/**',
                     '**/*.spec*',
                     '**/*.Tests/**'
                    ], cwd: project})
        .pipe(tap(file => {
            const contents = file.contents.toString();
            try {
                results = results.concat(matchCurlyBraces(contents)
                        .map(m => Object.assign(m, {path: file.path})));
            } catch (e) {
                results = results.concat([{
                    name: 'value',
                    value: file.contents.toString(),
                    start: 0,
                    end: file.contents.length,
                    error: e,
                    path: file.path
                }]);
            }
        }));
    });

    return importer.interpretAll([
        'task to promise',
    ])
    .then(r => {
        tasksToPromise = r[0].runInNewContext({gulp});
        return tasksToPromise(['find long functions']);
    })
    .then(() => {
        var code = results.map(r => r.value);
        var topLevel = results
        .filter(r => code
            .filter(c => c !== r.value && c.indexOf(r.value) > 0).length == 0);
        results = results
            .filter(m => topLevel.filter(c => c.value == m.value).length == 0);
        code = results.map(r => r.value);
        topLevel = topLevel.concat(results.filter(r => code
            .filter(c => c !== r.value && c.indexOf(r.value) > 0).length == 0));
        results = results
            .filter(m => topLevel.filter(c => c.value == m.value).length == 0);
        return results
            .filter(m => m.value.split(/\n/ig).length > 100)
    });
}
module.exports = findLongFunctions;
findLongFunctions;

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Documents/asm/subscription.identitymanagement';
$$.async();
findLongFunctions(project)
    .then(e => $$.sendResult(e.map(m => m.path)))
    .catch(e => $$.sendError(e));


```



Find relative paths and includes?

In [2]:
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = typeof project === 'undefined' ? path.join(PROFILE_PATH, 'Documents/portal') : project;
var importer = require('../Core');

var gulp = require('gulp');
var tap = require('gulp-tap');

var relatives = [];
var packages = [];
var missing = {};
var missingPackages = [];

gulp.task('get relative imports', function() {
    return gulp.src(['src/**/*.ts', '!**/*.spec.ts'], {cwd: project})
        .pipe(tap(function (file) {
            var content = file.contents.toString();
            var re = /from\s+["'](.*?)['"]/ig;
            var m;
            while ((m = re.exec(content))) {
                if(m[1].match(/^\./)) {
                    relatives[relatives.length] = file.path + '/' + m[1];
                    var res = path.resolve(path.join(path.dirname(file.path), m[1]));
                    if(!fs.existsSync(res) && !fs.existsSync(res + '.ts')) {
                        var key = path.basename(m[1]);
                        if(typeof missing[key] != 'undefined') {
                            missing[key][missing[key].length] = file.path + '/' + m[1];
                        } else {
                            missing[key] = [file.path + '/' + m[1]];
                        }
                    }
                } else {
                    packages[packages.length] = file.path + '/' + m[1];
                    try {
                        module.paths.unshift(project + '/node_modules');
                        require.resolve(m[1]);
                    } catch (e) {
                        if(missingPackages.indexOf(m[1]) == -1) {
                            missingPackages[missingPackages.length] = file.path + '/' + m[1];
                        }
                    } finally {
                        module.paths.shift();
                    }
                }
            }
        }));
});

var getRelativeImports = (p) => {
    relatives = [];
    packages = [];
    missing = {};
    missingPackages = [];
    project = p;
    return importer.interpretAll([
        'gulp task to a Promise'
    ])
        .then(r => r[0].runInNewContext({gulp}))
        .then(r => r(['get relative imports']))
        .then(r => ({relatives, packages, missing, missingPackages}));
}
module.exports = getRelativeImports;
getRelativeImports;



[Function: getRelativeLinks]

In [1]:
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = typeof project === 'undefined' ? path.join(PROFILE_PATH, 'Documents/universal') : project;
var importer = require('../Core');

$$.async();
importer.interpretAll([
    'relative paths and includes',
    'fix project paths'
])
.then(r => r[1].runInNEL())
.then(r => r['projectRelatives'](project))
//.then(r => console.log(r))
.then(r => $$.sendResult(r))
.catch(e => $$.sendError(e));



Search for **/core.module.ts

Replacing ../core/core.module -> ../../../imports/core.module in src/client/app/components/search.component.ts



{ mime: {} }